In [23]:
import numpy as np
import pandas as pd

In [24]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        if not isinstance(k, int) or k <= 0:
            raise ValueError("k must be a positive integer")
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y.astype(int)

    def predict(self, X, batch_size=1000):
        return self._predict_proba(X, batch_size).argmax(axis=1)

    def predict_proba(self, X, batch_size=1000):
        return self._predict_proba(X, batch_size)

    def _predict_proba(self, X, batch_size=1000):
        n_samples = X.shape[0]
        n_classes = len(np.unique(self.y_train))
        y_pred_proba = np.zeros((n_samples, n_classes))

        for i in range(0, n_samples, batch_size):
            batch = X[i:i+batch_size]
            if self.distance_metric == 'euclidean':
                distances = np.sqrt(((batch[:, np.newaxis, :] - self.X_train[np.newaxis, :, :]) ** 2).sum(axis=2))
            elif self.distance_metric == 'manhattan':
                distances = np.abs(batch[:, np.newaxis, :] - self.X_train[np.newaxis, :, :]).sum(axis=2)
            else:
                raise ValueError("Invalid distance metric")

            k = min(self.k, self.X_train.shape[0])
            k_indices = np.argpartition(distances, k, axis=1)[:, :k]
            k_nearest_labels = self.y_train[k_indices]

            for j, label in enumerate(k_nearest_labels):
                y_pred_proba[i+j] = np.bincount(label, minlength=n_classes) / k

        return y_pred_proba

In [25]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # Handle missing values
    train_data = train_data.dropna()
    test_data = test_data.dropna()

    # Encode categorical variables
    categorical_columns = ['Geography', 'Gender']
    for column in categorical_columns:
        train_data[column] = pd.Categorical(train_data[column]).codes
        test_data[column] = pd.Categorical(test_data[column]).codes

    # Separate features and target
    X_train = train_data.drop(['Exited', 'id', 'Surname'], axis=1)
    y_train = train_data['Exited'].astype(int)
    X_test = test_data.drop(['id', 'Surname'], axis=1)

    # Apply feature scaling
    X_train, X_test = feature_scaling(X_train, X_test)

    return X_train, y_train, X_test

In [26]:
def feature_scaling(X_train, X_test):
    # Min-Max scaling
    min_vals = X_train.min()
    max_vals = X_train.max()
    X_train_scaled = (X_train - min_vals) / (max_vals - min_vals)
    X_test_scaled = (X_test - min_vals) / (max_vals - min_vals)
    return X_train_scaled, X_test_scaled

In [27]:
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    n = len(X)
    n_test = int(n * test_size)
    indices = np.random.permutation(n)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]

    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    return X_train, X_test, y_train, y_test

In [28]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size = len(X) // n_splits
    scores = []

    for i in range(n_splits):
        start = i * fold_size
        end = (i + 1) * fold_size if i < n_splits - 1 else len(X)
        test_indices = indices[start:end]
        train_indices = np.concatenate([indices[:start], indices[end:]])

        X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
        y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

        knn.fit(X_train.values, y_train.values)
        y_pred = knn.predict(X_test.values)
        score = np.mean(y_pred == y_test)
        scores.append(score)

    return np.mean(scores), np.std(scores)

In [29]:
def evaluate_knn_with_different_k(X_train, y_train, X_val, y_val, k_values, distance_metrics):
    best_accuracy = 0
    best_k = 0
    best_metric = ''

    for metric in distance_metrics:
        for k in k_values:
            knn = KNN(k=k, distance_metric=metric)
            knn.fit(X_train, y_train)
            val_predictions = knn.predict(X_val)
            accuracy = np.mean(val_predictions == y_val)
            print(f"Validation Accuracy for k={k}, metric={metric}: {accuracy:.4f}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_k = k
                best_metric = metric

    print(f"\nBest k: {best_k}, Best metric: {best_metric} with accuracy: {best_accuracy:.4f}")
    return best_k, best_metric

In [30]:
# Load and preprocess data
X_train, y_train, X_test = preprocess_data('train.csv', 'test.csv')

# Split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [31]:
# Convert to numpy arrays for faster computation
X_train = X_train.values
y_train = y_train.values
X_val = X_val.values
y_val = y_val.values
X_test = X_test.values

In [32]:
# Example usage
k_values = [3, 5, 7, 9, 11]
distance_metrics = ['euclidean', 'manhattan']
best_k, best_metric = evaluate_knn_with_different_k(X_train, y_train, X_val, y_val, k_values, distance_metrics)

# Train with best parameters
knn = KNN(k=best_k, distance_metric=best_metric)
knn.fit(X_train, y_train)

Validation Accuracy for k=3, metric=euclidean: 0.8543
Validation Accuracy for k=5, metric=euclidean: 0.8633
Validation Accuracy for k=7, metric=euclidean: 0.8630
Validation Accuracy for k=9, metric=euclidean: 0.8640
Validation Accuracy for k=11, metric=euclidean: 0.8640
Validation Accuracy for k=3, metric=manhattan: 0.8533
Validation Accuracy for k=5, metric=manhattan: 0.8633
Validation Accuracy for k=7, metric=manhattan: 0.8640
Validation Accuracy for k=9, metric=manhattan: 0.8647
Validation Accuracy for k=11, metric=manhattan: 0.8633

Best k: 9, Best metric: manhattan with accuracy: 0.8647


In [33]:
# Perform cross-validation
cv_score, cv_std = cross_validate(pd.DataFrame(X_train), pd.Series(y_train), knn)
print(f"Cross-validation score: {cv_score:.4f} (+/- {cv_std:.4f})")

# Train on full dataset and make predictions on test set
full_X_train = np.vstack((X_train, X_val))
full_y_train = np.hstack((y_train, y_val))
knn.fit(full_X_train, full_y_train)

# Predict probabilities in batches to save memory
batch_size = 1000
test_predictions_proba = np.zeros((X_test.shape[0], 2))
for i in range(0, X_test.shape[0], batch_size):
    batch = X_test[i:i+batch_size]
    test_predictions_proba[i:i+batch_size] = knn.predict_proba(batch)

Cross-validation score: 0.8705 (+/- 0.0056)


In [34]:
# Save test predictions (probabilities for the positive class)
submission = pd.DataFrame({'id': pd.read_csv('test.csv')['id'], 'Exited': test_predictions_proba[:, 1]})
submission.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")

Submission file created: submission.csv
